# HW4

### Егоров Борис

`bwa index MG1655-K12.first10K.fasta`

`bwa index MG1655-K12.first400K.fasta`

`bwa mem -t 4 MG1655-K12.first10K.fasta ecoli_10K_err_1.fastq ecoli_10K_err_2.fastq > test.sam`

`bwa mem -t 4 MG1655-K12.first400K.fasta ecoli_400K_err_1.fastq ecoli_400K_err_2.fastq > ecoli.sam`

`java -jar ../Trimmomatic-0.39/trimmomatic-0.39.jar PE ecoli_10K_err_1.fastq ecoli_10K_err_2.fastq -baseout test_trimmed LEADING:20 TRAILING:20 SLIDINGWINDOW:10:20 MINLEN:20`

`java -jar ../Trimmomatic-0.39/trimmomatic-0.39.jar PE ecoli_400K_err_1.fastq ecoli_400K_err_2.fastq -baseout ecoli_trimmed LEADING:20 TRAILING:20 SLIDINGWINDOW:10:20 MINLEN:20`

`bwa mem -t 4 MG1655-K12.first10K.fasta test_trimmed_1P test_trimmed_2P > test_trimmed.sam`

``

| Reads | Tool        | Undetected | Detected & corrected | Detected & removed | Falsely corrected | Correctly unmodified | Incorrectly removed |
|:-----:|:-----------:| :---------:|:--------------------:|:------------------:|:-----------------:|:-----------------------:|:-------------------:|
| test  | trimmomatic |||||||
| ecoli | trimmomatic |||||||
| test  | bbduk |||||||
| ecoli | bbduk |||||||

